In [1]:
%matplotlib inline

In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from tqdm import tqdm_notebook

In [3]:
import torch
from torch import nn
from torch.nn import functional as F
from pyro import distributions as dist

In [4]:
import torch.utils.data.dataloader as dataloader
import torchvision
from torchvision import transforms
from torchvision.datasets import MNIST

In [5]:
import pyro
import deepppl
import os

In [6]:
sns.set_style('whitegrid')


### Basic example showing the interface of *DeepPPL*
This example uses a NN: `MLP` adding uncertainity to its `parameters`

The `DeepPPL` model should be built with the `mlp` as keyword argument


```stan
data {
    int batch_size;
    int <lower=0, upper=1> imgs[28,28,batch_size]; 
    int <lower=0, upper=10>  labels[batch_size];
}

networks {
    MLP mlp with parameters:
        l1.weight;
        l1.bias;
        l2.weight;
        l2.bias;
}

prior
{
    mlp.l1.weight ~  normal(zeros(mlp.l1.weight$shape), ones(mlp.l1.weight$shape));
    mlp.l1.bias ~ normal(zeros(mlp.l1.bias$shape), ones(mlp.l1.bias$shape));
    mlp.l2.weight ~ normal(zeros(mlp.l2.weight$shape), ones(mlp.l2.weight$shape));
    mlp.l2.bias ~  normal(zeros(mlp.l2.bias$shape), ones(mlp.l2.bias$shape));
}

guide parameters
{
    real l1wloc[mlp.l1.weight$shape];
    real l1wscale[mlp.l1.weight$shape];
    real l1bloc[mlp.l1.bias$shape];
    real l1bscale[mlp.l1.bias$shape];
    real l2wloc[mlp.l2.weight$shape];
    real l2wscale[mlp.l2.weight$shape];
    real l2bloc[mlp.l2.bias$shape];
    real l2bscale[mlp.l2.bias$shape];
}

guide {
    l1wloc = randn(l1wloc$shape);
    l1wscale = randn(l1wscale$shape);
    mlp.l1.weight ~  normal(l1wloc, softplus(l1wscale));
    l1bloc = randn(l1bloc$shape);
    l1bscale = randn(l1bscale$shape);
    mlp.l1.bias ~ normal(l1bloc, softplus(l1bscale));
    l2wloc = randn(l2wloc$shape);
    l2wscale = randn(l2wscale$shape);
    mlp.l2.weight ~ normal(l2wloc, softplus(l2wscale));
    l2bloc = randn(l2bloc$shape);
    l2bscale = randn(l2bscale$shape);
    mlp.l2.bias ~ normal(l2bloc, softplus(l2bscale));
}

model {
    real logits[batch_size];
    logits = mlp(imgs);
    labels ~ categorical_logits(logits);
}
```

In [7]:
batch_size, nx, nh, ny = 128, 28 * 28, 1024, 10

In [8]:
# Model

class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.l1 = torch.nn.Linear(nx, nh)
        self.l2 = torch.nn.Linear(nh, ny)
        self.relu = torch.nn.ReLU()

    def forward(self, x):
        h = self.relu(self.l1(x.view((-1, nx))))
        yhat = self.l2(h)
        return F.log_softmax(yhat, dim=-1)

mlp = MLP()


In [9]:
def loadData(batch_size):
    train = MNIST(os.environ.get("DATA_DIR", '.') + "/data", train=True, download=True, transform=transforms.Compose([
        transforms.ToTensor(),  # ToTensor does min-max normalization.
    ]), )
    test = MNIST(os.environ.get("DATA_DIR", '.') + "/data", train=False, download=True, transform=transforms.Compose([
        transforms.ToTensor(),  # ToTensor does min-max normalization.
    ]), )
    dataloader_args = dict(shuffle=True, batch_size=batch_size,
                        num_workers=3, pin_memory=False)
    train_loader = dataloader.DataLoader(train, **dataloader_args)
    test_loader = dataloader.DataLoader(test, **dataloader_args)
    return train_loader, test_loader

In [10]:
def categorical_logits(logits):
    return dist.Categorical(logits=logits)

def predict(data, posterior):
    predictions = [model(data) for model in posterior]
    prediction = torch.stack(predictions).mean(dim=0)
    return prediction.argmax(dim=-1)


##### Build `DpPPPL` model

In [11]:
train_loader, test_loader = loadData(batch_size)
model = deepppl.PyroModel(model_file = '../tests/good/mlp.stan', mlp=mlp, categorical_logits=categorical_logits)
svi = model.svi(params = {'lr' : 0.01})

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw




Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw
Processing...
Done!
ANTLR runtime and generated code versions disagree: 4.7.2!=4.8
ANTLR runtime and generated code versions disagree: 4.7.2!=4.8


../torch/csrc/utils/tensor_numpy.cpp:141: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program.


#### Train the model using `svi`

In [12]:
for epoch in range(2):  # loop over the dataset multiple times
    for j, (imgs, lbls) in enumerate(train_loader, 0):
        # calculate the loss and take a gradient step
        loss = svi.step(batch_size, imgs, lbls)
        if (j+1) % 100 == 0:
            print('Epoch:{} Iteration:{} Loss:{}'.format(epoch, j, loss))


/anaconda3/lib/python3.7/site-packages/pyro/primitives.py:406: FutureWarning: The `random_module` primitive is deprecated, and will be removed in a future release. Use `pyro.nn.Module` to create Bayesian modules from `torch.nn.Module` instances.
  "modules from `torch.nn.Module` instances.", FutureWarning)


Epoch:0 Iteration:99 Loss:407706.9223251343
Epoch:0 Iteration:199 Loss:234124.97799873352
Epoch:0 Iteration:299 Loss:152196.09954833984
Epoch:0 Iteration:399 Loss:107269.11954212189
Epoch:1 Iteration:99 Loss:61747.872787475586
Epoch:1 Iteration:199 Loss:48487.588163375854
Epoch:1 Iteration:299 Loss:36934.18068790436
Epoch:1 Iteration:399 Loss:35177.457597732544


##### Compute a posterior distribution
In this case, the distribution is a distribution over possible MLPs. 
Each MLP will give a prediction and the uncertainity can be seen in the distribution of the predicted values

In [13]:
posterior = svi.posterior(30)

#### Prediction
For each element in the testset, we expect the accuracy to be higher than 80%

In [14]:
for j, data in enumerate(test_loader):
    images, labels = data
    accuracy = (predict(images, posterior) == labels).type(torch.float).mean()
    assert accuracy > 0.8

And a single batch can be predicted

In [15]:
predict(images, posterior)

tensor([5, 8, 0, 9, 9, 2, 9, 7, 2, 1, 5, 0, 5, 7, 7, 0],
       grad_fn=<NotImplemented>)